In [1]:
!pip install fpdf

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable


In [42]:
import os
import shutil
import numpy as np
import pandas as pd
import calendar
from datetime import datetime
from fpdf import FPDF
from datetime import date
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False

Создаем 2 случайных датасета

In [165]:
df1 = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))
df2 = pd.DataFrame(np.random.randint(0,100,size=(102, 5)), columns=list('ABCDE'))

In [180]:
class PDF(FPDF):
    def __init__(self):
        super().__init__()
        self.WIDTH = 210
        self.HEIGHT = 297
        
    def header(self):
        # Custom logo and positioning
        # Create an `assets` folder and put any wide and short image inside
        # Name the image `logo.png`
        self.image('logo.png', 10, 8, 33)
        self.set_font('Arial', 'B', 11)
        self.cell(self.WIDTH - 80)
        self.cell(60, 1, 'Daqapy Report', 0, 1, 'R')
        
        self.set_font('Arial', '', 7)    
        self.set_text_color(128)
        self.cell(0, 7, 'https://github.com/falconwatch/daqapy', 0, 0, 'R')
        self.ln(20)
        
    def footer(self):
        # Page numbers in the footer
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.set_text_color(128)
        self.cell(0, 10, 'Page ' + str(self.page_no()), 0, 0, 'C')

    def page_body(self, df1, df2):

        # data preparation
        df1_temp = df1.describe().transpose()[["count", "mean", "std"]]
        df2_temp = df2.describe().transpose()[["count", "mean", "std"]]
        df3 = pd.merge(df1_temp, df2_temp, left_index=True, right_index=True, suffixes=(' ','  '))
        df3 = df3.round({'std ': 2, 'std  ': 2, 'mean ': 2, 'mean  ': 2})
        df3['AI comment'] = 'ok'
        
        if (set(df1.columns) == set(df2.columns)):
            col_comparisson = 'the same columns'
        else:
            col_comparisson = 'different columns'

        
        # Determine how many plots there are per page and set positions
        # and margins accordingly
        self.set_font('Arial','',9)
        #df_name =[x for x in globals() if globals()[x] is df][0]
        df_name = '232'
        today = date.today()
        self.cell(10,1,"Report date: %s." % today)
        self.ln(10)
        # table1
        self.cell(110,6,"",0,0,'C')
        self.cell(40,6,"Control Dataset",1,0,'C')
        self.cell(40,6,"Test Dataset",1,1,'C') 
        self.cell(110,6,"Number of rows",1,0,'L')
        self.cell(40,6,str(df1.shape[0]),1,0,'C')
        self.cell(40,6,str(df2.shape[0]),1,1,'C')    
        self.cell(110,6,"Number of columns",1,0,'L')
        self.cell(40,6,str(df1.shape[1]),1,0,'C')
        self.cell(40,6,str(df2.shape[1]),1,1,'C')         
        self.ln(10)
        # table1
        self.cell(110,6,"Attribute composition of the control and test sample by field names",1,0,'L')
        self.cell(80,6,col_comparisson,1,1,'C')        
        self.ln(10)        
        # table3 
        self.cell(50,6,"",1,0,'C')
        self.cell(60,6,"Control Dataset",1,0,'C')
        self.cell(80,6,"Test Dataset",1,1,'C')      
        self.cell(50,6,"Columns",1,0,'L')
        for c in df3.columns:
          newrow = 0
          if c == 'AI comment': #new line after column with name std'
              newrow = 1
          self.cell(20,6,c,1,newrow,'C')

        for roq in df3.iterrows():
          index = roq[0]
          values = roq[1]
          self.cell(50,6,index,1,0,'L')
          for i in range(len(values)):
            newrow = 0
            if type(values.values[i]) == pd.core.series.Series:
              printed_value = str("{:.2f}".format(values.values[i]))
            else:
              printed_value = str(values.values[i])
            if i == len(values)-1:
              newrow = 1
            self.cell(20,6,printed_value,1,newrow,'C')

        self.ln(20)                
            
    def print_page(self, df1, df2):
        # Generates the report
        self.add_page()
        self.page_body(df1, df2)

In [181]:
pdf = PDF()
pdf.print_page(df1, df2)
pdf.output('daqareport.pdf', 'F')

''